In [7]:
import requests
import time
import json

match_id = "133462"
api_url = f"https://www.cricbuzz.com/api/cricket-match/commentary/{match_id}"

headers = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json, text/plain, */*",
    "Referer": "https://www.cricbuzz.com/"
}

all_data = []
last_ball = None  

while True:
    response = requests.get(api_url, headers=headers)
    data = response.json()
    miniscore = data.get("miniscore", {})
    commentary_list = data.get("commentaryList", [])

    if commentary_list:
        latest = commentary_list[0]
        ball_id = f"{latest.get('overNumber')}-{latest.get('ballNbr')}"
        comm_text = latest.get("commText", "")

        if ball_id != last_ball and latest.get("overNumber") is not None:
            scorecard = {}

            # ✅ Get scorecard directly from JSON
            bat_team = miniscore.get("batTeam", {}).get("teamName")
            innings = miniscore.get("batTeam", {}).get("inningsScoreList", [])
            if innings:
                runs = innings[0].get("score")
                wkts = innings[0].get("wkts")
                overs = innings[0].get("overs")
                scorecard["current_innings"] = f"{bat_team} {runs}-{wkts} ({overs} Ov)"

            # Batters
            batters = []
            for b in [miniscore.get("batsmanStriker"), miniscore.get("batsmanNonStriker")]:
                if b:
                    batters.append({
                        "name": b.get("batName"),
                        "runs": b.get("batRuns"),
                        "balls": b.get("batBalls"),
                        "fours": b.get("batFours"),
                        "sixes": b.get("batSixes"),
                        "sr": b.get("batStrikeRate")
                    })
            if batters:
                scorecard["batters"] = batters

            # Bowlers
            bowlers = []
            for bowler in [miniscore.get("bowlerStriker"), miniscore.get("bowlerNonStriker")]:
                if bowler:
                    bowlers.append({
                        "name": bowler.get("bowlName"),
                        "overs": bowler.get("bowlOvs"),
                        "runs": bowler.get("bowlRuns"),
                        "wkts": bowler.get("bowlWkts"),
                        "eco": bowler.get("bowlEcon")
                    })
            if bowlers:
                scorecard["bowlers"] = bowlers

            # Final entry
            entry = {
                "over": latest.get("overNumber"),
                "ball": latest.get("ballNbr"),
                "commentary": comm_text,
                "scorecard": scorecard
            }

            all_data.append(entry)

            # Save to JSON
            with open("Live_match_data.json", "w") as f:
                json.dump(all_data, f, indent=4)

            print(f"✅ Saved: Over {entry['over']}  b:{entry['ball']} - {comm_text}")
            last_ball = ball_id

    time.sleep(10)


✅ Saved: Over 18.6  b:114 - Swepson to Sam Harper, no run
✅ Saved: Over 19.3  b:117 - Jack Wildermuth to Handscomb, no run
✅ Saved: Over 19.6  b:120 - Jack Wildermuth to Handscomb, 2 runs, to deep mid-wicket
✅ Saved: Over 20.2  b:122 - Swepson to Handscomb, 1 run, to long-on
✅ Saved: Over 20.4  b:124 - Swepson to Sam Harper, 1 run, to long-off
✅ Saved: Over 20.6  b:126 - Swepson to Sam Harper, no run
✅ Saved: Over 21.5  b:131 - Jack Clayton to Sam Harper, 1 run, to mid-off
✅ Saved: Over 21.6  b:132 - Jack Clayton to Handscomb, 1 run, to deep backward point
✅ Saved: Over 22.3  b:135 - Swepson to Sam Harper, no run
✅ Saved: Over 22.5  b:137 - Swepson to Handscomb, 1 run, to mid-off
✅ Saved: Over 22.6  b:138 - Swepson to Sam Harper, no run


KeyboardInterrupt: 

In [23]:
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0"}

urls = {
    "Live": "https://www.cricbuzz.com/cricket-match/live-scores",
    "Completed": "https://www.cricbuzz.com/cricket-match/live-scores/recent-matches",
    "Upcoming": "https://www.cricbuzz.com/cricket-match/live-scores/upcoming-matches"
}

matches = []

for status, url in urls.items():
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Each match card is under "cb-mtch-lst"
    match_cards = soup.find_all("div", class_="cb-mtch-lst")

    for card in match_cards:
        # Match title
        title_tag = card.find("a", class_="text-hvr-underline")
        if not title_tag:
            continue

        match_name = title_tag.get_text(strip=True)
        match_link = "https://www.cricbuzz.com" + title_tag["href"]

        matches.append({
            "match": match_name,
            "status": status,
            "link": match_link
        })

# Print results
for m in matches:
    print(f"{m['match']}  --> {m['status']}  ({m['link']})")


Guyana Amazon Warriors vs Saint Lucia Kings,  --> Live  (https://www.cricbuzz.com/live-cricket-scores/116819/gaw-vs-slk-qualifier-1-caribbean-premier-league-2025)
Yorkshire vs Sussex,  --> Live  (https://www.cricbuzz.com/live-cricket-scores/113235/yorks-vs-sus-63rd-match-county-championship-division-one-2025)
Essex vs Warwickshire,  --> Live  (https://www.cricbuzz.com/live-cricket-scores/113226/ess-vs-warks-61st-match-county-championship-division-one-2025)
Nottinghamshire vs Surrey,  --> Live  (https://www.cricbuzz.com/live-cricket-scores/113244/notts-vs-sur-64th-match-county-championship-division-one-2025)
Worcestershire vs Durham,  --> Live  (https://www.cricbuzz.com/live-cricket-scores/113229/worcs-vs-dur-62nd-match-county-championship-division-one-2025)
Somerset vs Hampshire,  --> Live  (https://www.cricbuzz.com/live-cricket-scores/113253/som-vs-ham-65th-match-county-championship-division-one-2025)
Leicestershire vs Kent,  --> Live  (https://www.cricbuzz.com/live-cricket-scores/113

In [26]:
import requests
from bs4 import BeautifulSoup
import json

headers = {"User-Agent": "Mozilla/5.0"}

urls = {
    "Live": "https://www.cricbuzz.com/cricket-match/live-scores",
    "Completed": "https://www.cricbuzz.com/cricket-match/live-scores/recent-matches",
    "Upcoming": "https://www.cricbuzz.com/cricket-match/live-scores/upcoming-matches"
}

matches = []

for status, url in urls.items():
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Each match card
    match_cards = soup.find_all("div", class_="cb-mtch-lst")

    for card in match_cards:
        # ✅ Full match title is inside this link
        title_tag = card.find("a", class_="text-hvr-underline")
        if not title_tag:
            continue

        match_name = title_tag.get_text(strip=True)
        match_link = "https://www.cricbuzz.com" + title_tag["href"]

        matches.append({
            "match": match_name,
            "status": status,
            "link": match_link
        })

# ✅ Save to JSON file
with open("matches_list.json", "w", encoding="utf-8") as f:
    json.dump(matches, f, indent=4)

print("✅ Matches saved to matches_list.json")


✅ Matches saved to matches_list.json
